In [1]:


from utils.graphs import compile_graph


/opt/anaconda3/envs/ai-playground/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/ai-playground/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/anaconda3/envs/ai-playground/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
research_chain = compile_graph()

/Users/zding/Documents/aibootcamp/code/AIE4-DemoDay/utils/agent_helper.py:78: LangChainDeprecationWarning: The method `BaseChatOpenAI.bind_functions` was deprecated in langchain-openai 0.2.1 and will be removed in 0.3.0. Use :meth:`~langchain_openai.chat_models.base.ChatOpenAI.bind_tools` instead.
  | llm.bind_functions(functions=[function_def], function_call="route")


Processing data/literature/s41591-023-02278-8.pdf...
[                                        ] (0/3[=                                       ] ( 1/3[==                                      ] ( 2/35[===                                     ] ( 3/3[====                                    ] ( 4/35[=====                                   ] ( 5/3[======                                  ] ( 6/35[========                                ] ( 7/35=[=========                               ] ( 8/3[==========                              ] ( 9/35[===========                             ] (10/3[============                            ] (11/35[=============                           ] (12/3[==============                          ] (13/35[================                        ] (14/35=[=================                       ] (15/3[==================                      ] (16/35[===================                     ] (17/3[====================                    ] (18/35[=====================  

In [5]:
for s in research_chain.stream(
    "what caused diabetes?", {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'Supervisor': {'next': 'ScholarQuery'}}
---
{'Query': {'messages': [HumanMessage(content='The research paper titled "Social Determinants of Health and Structural Inequities-Root Causes of Diabetes Disparities" by Michelle Ogunwole and S. Golden discusses how social determinants of health (SDOH) and structural inequities are root causes of diabetes disparities. The paper highlights the disproportionate burden of diabetes on historically marginalized groups and emphasizes the importance of addressing contextual factors outside of traditional medical care to improve outcomes, especially for marginalized communities. The authors argue that social and environmental factors, collectively known as SDOH, play a significant role in shaping health outcomes and contributing to health disparities. Addressing SDOH at the structural and systems level is crucial for achieving health equity and reducing diabetes disparities.', additional_kwargs={}, response_metadata={}, name='ScholarQuery')]}}
---
{'

In [4]:
from IPython.display import Image, display

try:
    display(Image(research_chain.get_graph(xray=True).draw_mermaid_png()))
except:
    print('Na...')
    pass

Na...
